**Data Mining Project, midterm 2021/2022**
**Authors:** Niko Dalla Noce, Alessandro Ristori, Giuseppe Lombardi
**Date:**




#**Task 1: Data Understanding and Data Preparation**

**Importing libraries**

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import scipy

import zipfile
with zipfile.ZipFile("prj_data.zip", 'r') as zip_ref:
     zip_ref.extractall()

**Load the datasets**

Read the data from the three csv files, each one is assigned to a different datafram for now.

In [6]:
df_male = pd.read_csv("dataset/male_players.csv", sep=",")
df_female = pd.read_csv("dataset/female_players.csv", sep=",")
df_matches = pd.read_csv("dataset/tennis_matches.csv", sep=",")
df_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186128 entries, 0 to 186127
Data columns (total 50 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Unnamed: 0          186128 non-null  int64  
 1   tourney_id          186073 non-null  object 
 2   tourney_name        186103 non-null  object 
 3   surface             185940 non-null  object 
 4   draw_size           186099 non-null  float64
 5   tourney_level       186099 non-null  object 
 6   tourney_date        186100 non-null  float64
 7   match_num           186101 non-null  float64
 8   winner_id           186073 non-null  float64
 9   winner_entry        25827 non-null   object 
 10  winner_name         186101 non-null  object 
 11  winner_hand         186082 non-null  object 
 12  winner_ht           49341 non-null   float64
 13  winner_ioc          186099 non-null  object 
 14  winner_age          183275 non-null  float64
 15  loser_id            186100 non-nul

Let's take a look to the players dataframes by calling the head() method, which shows the first five lines of the dataframes.

In [ ]:
df_male.head()

,name,surname
0,Gardnar,Mulloy
1,Pancho,Segura
2,Frank,Sedgman
3,Giuseppe,Merlo
4,Richard Pancho,Gonzales


In [ ]:
df_female.head()

,name,surname
0,Bobby,Riggs
1,X,X
2,Martina,Hingis
3,Mirjana,Lucic
4,Justine,Henin


Let's now use the info() method to obtain information on the two player datasets.

In [ ]:
print("Male players dataframe info")
df_male.info()
print("\nFemale players dataframe info")
df_female.info()

Male players dataframe info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55208 entries, 0 to 55207
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   name     55031 non-null  object
 1   surname  55166 non-null  object
dtypes: object(2)
memory usage: 862.8+ KB

Female players dataframe info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46172 entries, 0 to 46171
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   name     44505 non-null  object
 1   surname  46172 non-null  object
dtypes: object(2)
memory usage: 721.6+ KB


As we can see from the previous infos, there are null values both in the male and female dataset.

In [ ]:
print("Male players dataframe null values:\n{0}".format(df_male.isnull().any()))
print("\nFemale players dataframe null values:\n{0}".format(df_female.isnull().any()))

Male players dataframe null values:
name       True
surname    True
dtype: bool

Female players dataframe null values:
name        True
surname    False
dtype: bool


We need to drop such elements with null values.

In [ ]:
df_male = df_male.dropna()
df_female = df_female.dropna()
print("Male players dataframe info:")
df_male.info()
print("\nFemale players dataframe info:")
df_female.info()

Male players dataframe info:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 55031 entries, 0 to 55207
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   name     55031 non-null  object
 1   surname  55031 non-null  object
dtypes: object(2)
memory usage: 1.3+ MB

Female players dataframe info:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 44505 entries, 0 to 46171
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   name     44505 non-null  object
 1   surname  44505 non-null  object
dtypes: object(2)
memory usage: 1.0+ MB


There could be duplicates in the same dataset, but we know that there's a possibility that two players could share the same name and surname, so we won't change anything.

But, there are elements where the values are missing, or incomplete, but not null (look at df_female.head()), we need to remove such elements.

In [ ]:
elements_not_to_keep = ["Unknown", "??"]
df_male = df_male[df_male["name"].str.len()>1]
df_male = df_male[df_male["surname"].str.len()>1]
df_male = df_male[~df_male["name"].isin(elements_not_to_keep)]
df_male = df_male[~df_male["surname"].isin(elements_not_to_keep)]
df_male.info()
# df_male[df_male["name"].isin(["??"])]
# df_male.where(df_male.name.str.len()==1)
# Ci sono altri elementi che non sono compatibili, andrebbero visti a mano

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53965 entries, 0 to 55207
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   name     53965 non-null  object
 1   surname  53965 non-null  object
dtypes: object(2)
memory usage: 1.2+ MB
